In [4]:
"""
DSC 106 – AGAINST visuals (final polish)
This version positions labels intuitively at the START of each line: 
'CO₂' near the first black point, and 'Renewable share' near the first blue point.
"""

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.patheffects as pe

# Formatter helpers
def _fmt_pct(x, pos):
    return f"{x:.0f}%"

def _fmt_gt(x, pos):
    return f"{x:.0f}"

def load_imf_renewables(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[df["Indicator"].str.contains("Electricity Generation", case=False, na=False)]
    df = df[df["Energy_Type"].isin(["Total Renewable", "Total Non-Renewable"])]
    year_cols = [c for c in df.columns if c.startswith("F") and c[1:].isdigit()]
    long = df.melt(id_vars=["Country","ISO3","Energy_Type","Unit"],
                   value_vars=year_cols, var_name="FYear", value_name="Value")
    long["Year"] = long["FYear"].str[1:].astype(int)
    long = long.drop(columns=["FYear"]).dropna(subset=["Value"])
    long = long[(long["Year"] >= 2000) & (long["Year"] <= 2024)]
    pvt = long.pivot_table(index=["Country","ISO3","Year"], columns="Energy_Type", values="Value", aggfunc="sum").reset_index()
    pvt = pvt.rename(columns={"Total Renewable":"gen_renew_gwh","Total Non-Renewable":"gen_nonrenew_gwh"})
    pvt["gen_total_gwh"] = pvt[["gen_renew_gwh","gen_nonrenew_gwh"]].sum(axis=1)
    pvt["renew_share"] = pvt["gen_renew_gwh"] / pvt["gen_total_gwh"]
    return pvt

def compute_global_series_dummy(renew: pd.DataFrame):
    g = renew.groupby("Year").agg(gen_renew_gwh=("gen_renew_gwh","sum"), gen_total_gwh=("gen_total_gwh","sum")).reset_index()
    g["renew_share_global"] = g["gen_renew_gwh"] / g["gen_total_gwh"]
    co2 = pd.DataFrame({"Year": np.arange(2000, 2025), "co2_gt": np.linspace(25, 33, 25)})
    return g, co2

def plot_visual_3(global_renew: pd.DataFrame, global_co2: pd.DataFrame, outpath: Path):
    df = pd.merge(global_co2[["Year","co2_gt"]], global_renew[["Year","renew_share_global"]], on="Year", how="inner")

    fig, ax1 = plt.subplots(figsize=(10, 5.6))
    fig.patch.set_facecolor("white")

    co2_line = "#303030"; co2_fill = "#e1e4e8"; ren_line = "#1f77b4"

    ax1.fill_between(df["Year"], 0, df["co2_gt"], color=co2_fill, alpha=1.0)
    ax1.plot(df["Year"], df["co2_gt"], color=co2_line, lw=2.6, solid_capstyle="round")
    ax1.set_ylabel("Global CO₂ (Gt)", color=co2_line)
    ax1.tick_params(axis='y', colors=co2_line)
    ax1.yaxis.set_major_formatter(FuncFormatter(_fmt_gt))
    ax1.set_ylim(0, df["co2_gt"].max()*1.1)

    ax2 = ax1.twinx()
    ax2.plot(df["Year"], df["renew_share_global"]*100, color=ren_line, lw=2.8, solid_capstyle="round")
    ax2.set_ylabel("Renewable share of electricity (%)", color=ren_line)
    ax2.tick_params(axis='y', colors=ren_line)
    ax2.yaxis.set_major_formatter(FuncFormatter(_fmt_pct))
    ax2.set_ylim(0, (df["renew_share_global"].max()*100)*1.1)

    halo = [pe.withStroke(linewidth=3.5, foreground="white")]

    # Starting points (label there)
    x_start = df["Year"].iloc[0]
    y_co2_start = df["co2_gt"].iloc[0]
    y_ren_start = df["renew_share_global"].iloc[0]*100

    # Labels: CO2 above its first point, Renewable share below its first point
    ax1.text(x_start, y_co2_start + 0.8, "CO₂", color=co2_line, fontsize=12, weight='bold',
             ha='left', va='bottom', path_effects=halo)
    ax2.text(x_start, y_ren_start - 1.2, "Renewable share", color=ren_line, fontsize=12, weight='bold',
             ha='left', va='top', path_effects=halo)

    # End markers for continuity
    ax1.scatter([x_start, df["Year"].iloc[-1]], [y_co2_start, df["co2_gt"].iloc[-1]], color=co2_line, s=28, zorder=3)
    ax2.scatter([x_start, df["Year"].iloc[-1]], [y_ren_start, df["renew_share_global"].iloc[-1]*100], color=ren_line, s=28, zorder=3)

    ax1.set_title("Despite Rising Renewable Share, Global CO₂ Continues to Increase (2000–2024)")
    ax1.set_xlabel("Year")
    ax1.grid(True, axis="y", alpha=0.22)

    fig.tight_layout()
    fig.savefig(outpath, dpi=320, bbox_inches='tight')
    plt.close(fig)

# Main
if __name__ == "__main__":
    DATA_DIR = Path("../data")
    PLOTS_DIR = Path("../plots")
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)
    IMF_RENEWABLE_CSV = DATA_DIR / "Renewable_Energy.csv"

    renew = load_imf_renewables(IMF_RENEWABLE_CSV)
    gRenew, gCO2 = compute_global_series_dummy(renew)

    out = PLOTS_DIR / "visual3_against_global_co2_vs_renewshare.png"
    plot_visual_3(gRenew, gCO2, out)
    print(f"Saved: {out}")


Saved: ../plots/visual3_against_global_co2_vs_renewshare.png
